<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/TrajectoryOfSphereWithDragCrisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [2]:
def Cd(Re):
    a=Re/5
    b=Re/2.63e5
    c=Re/1e6
    return 24/Re + 2.6*a/(1+a**1.52) +0.411*b**(-25)/(1+b**(-25)) + 0.25*c/(1+c)

In [3]:
x = np.logspace(2,7,500)
Cd_vals = Cd(x)
fig = make_subplots()
fig.add_scatter(x=x,y=Cd_vals, mode='lines')
fig.update_xaxes(type='log')
fig.update_layout(width=800,height=600, template='plotly_dark')

In [32]:
rho = 1.29 #kg/m3
mu = 1.8e-5
d = 0.22
A = np.pi*(d**2)/4
g = 9.81
m= 0.42
s=np.array([0, 0, 60])

def rhs(t, xv):
    v=xv[3:]
    v_mag = np.linalg.norm(v)
    Re = rho*v_mag*d/mu
    dxdt = v
    Cs = 0.2
    dvdt = np.array([0,0, -g]) - rho*Cd(Re)*A*v_mag/2*v/m + Cs * rho * A *d/2* np.cross(s,v)/m
    return np.concatenate([dxdt,dvdt])

def event(t, xv):
    return xv[2]

event.terminal=True
event.direction=-1

In [39]:
v_mag_mph = 70
phi = 15 * np.pi/180
theta = 30 * np.pi/180
v_mag = v_mag_mph * 0.44704
v0 = v_mag*np.array([np.cos(theta)*np.sin(phi), np.cos(theta)*np.cos(phi), np.sin(theta)])

In [40]:
tend=10
res = solve_ivp(rhs, (0,tend), np.concatenate([[0,0, 0],v0]), method='Radau', dense_output=True, events=event)

In [41]:
t_plot = np.linspace(0,res.t_events[0][0],50)
x,y,z =res.sol(t_plot)[:3]
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'surface'}]])
fig.add_scatter3d(x=x,y=y,z=z,mode='lines',row=1,col=1)
fig.update_layout(width=600,height=600,template='plotly_dark',
                  scene=dict(camera=dict(eye=dict(x=0, y=-1, z=0.2)), 
           xaxis=dict(range=[-5,15]),
           yaxis=dict(range=[0,100]),
           zaxis=dict(range=[0,50]),
           aspectmode='manual',
           aspectratio=dict(x=0.5, y=1, z=0.5)
           )
)